In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.0.9-py3-none-any.whl size=184112 sha256=8b2f25116b5526ae511ce33289f9ca0e5e3a7055da386b2fa01ea41b9fad9126
  Stored in directory: /root/.cache/pip/wheels/1b/9c/b8/9759d7cc64a1e01bb9872ade80cb7db445ccf506e083325106
Successfully built pyod


In [3]:
import random
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

from pyod.models.abod import ABOD

from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import *

from pyod.models.abod import ABOD
from pyod.models.deep_svdd import DeepSVDD
from sklearn.ensemble import IsolationForest

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers import Input, Dense, BatchNormalization, Dropout, LSTM, TimeDistributed, RepeatVector
from keras import regularizers
from keras.models import Model,Sequential
from keras.callbacks import EarlyStopping
from keras.backend import clear_session

import pickle
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Random Seed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(69) # Seed 고정

## Data Load

In [37]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI_SPARK/Data/train_data.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI_SPARK/Data/test_data.csv')

## Pre-Processing

#### HP

In [38]:
train.loc[train['type'] == 0, 'HP'] = 30
train.loc[train['type'] == 1, 'HP'] = 20
train.loc[train['type'] == 2, 'HP'] = 10
train.loc[train['type'] == 3, 'HP'] = 50
train.loc[train['type'] == 4, 'HP'] = 30
train.loc[train['type'] == 5, 'HP'] = 30
train.loc[train['type'] == 6, 'HP'] = 30
train.loc[train['type'] == 7, 'HP'] = 30

test.loc[test['type'] == 0, 'HP'] = 30
test.loc[test['type'] == 1, 'HP'] = 20
test.loc[test['type'] == 2, 'HP'] = 10
test.loc[test['type'] == 3, 'HP'] = 50
test.loc[test['type'] == 4, 'HP'] = 30
test.loc[test['type'] == 5, 'HP'] = 30
test.loc[test['type'] == 6, 'HP'] = 30
test.loc[test['type'] == 7, 'HP'] = 30

#### kw

In [39]:
train['kw'] = train['HP'] * 0.746
test['kw'] = test['HP'] * 0.746

#### 무지성 생성

In [40]:
cols = list(train)
del cols[cols.index('type')]
del cols[cols.index('out_pressure')]

num = 0

for col_1 in cols:
  for col_2 in cols:
    train['col_mul_'+str(num)] = train[col_1] * train[col_2]
    train['col_div_'+str(num)] = train[col_1] / train[col_2]
    num += 1


In [ ]:
for col in list(train):
  print(train[col].unique())

In [43]:
train

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type,HP,kw,...,col_mul_59,col_div_59,col_mul_60,col_div_60,col_mul_61,col_div_61,col_mul_62,col_div_62,col_mul_63,col_div_63
count,2463.000000,2463.000000,2.463000e+03,2463.000000,2463.000000,2463.000000,2463.000000,2463.000000,2463.000000,2463.000000,...,2463.000000,2463.000000,2463.000000,2463.000000,2463.000000,2463.000000,2463.000000,2.463000e+03,2463.000000,2463.0
mean,2.128843,49.001137,7.000000e-01,27.490715,2372.576939,65.766707,3.452972,2.957369,28.014616,20.898904,...,49834.508258,0.009630,1386.897747,0.319348,75.609962,6.107047,675.125457,7.460000e-01,503.643591,1.0
std,1.202328,8.725091,1.110448e-16,13.604597,687.870774,7.797152,1.576386,2.230113,10.964670,8.179644,...,25429.918968,0.004942,594.526700,0.125326,46.373925,1.813494,500.858361,1.110448e-16,373.640337,0.0
min,0.330000,32.030000,7.000000e-01,5.340000,1201.000000,50.360000,1.970000,0.000000,10.000000,7.460000,...,8966.920000,0.002094,375.685600,0.097402,14.696200,0.682213,74.600000,7.460000e-01,55.651600,1.0
25%,1.280000,41.600000,7.000000e-01,17.110000,1784.500000,59.150000,2.880000,1.000000,20.000000,14.920000,...,30865.750000,0.006457,1024.034200,0.249186,50.429600,5.135988,298.400000,7.460000e-01,222.606400,1.0
50%,1.960000,48.980000,7.000000e-01,25.850000,2358.000000,65.640000,3.260000,3.000000,30.000000,22.380000,...,46751.820000,0.008695,1393.602600,0.317492,70.049400,6.357955,671.400000,7.460000e-01,500.864400,1.0
75%,2.820000,56.355000,7.000000e-01,36.000000,2971.000000,72.275000,3.730000,5.000000,30.000000,22.380000,...,66114.250000,0.012241,1655.672400,0.380515,83.253600,7.474950,671.400000,7.460000e-01,500.864400,1.0
max,6.220000,64.960000,7.000000e-01,72.000000,3564.000000,80.520000,21.870000,7.000000,50.000000,37.300000,...,132676.100000,0.030980,3003.396000,0.685914,326.300400,9.638243,1865.000000,7.460000e-01,1391.290000,1.0


In [ ]:
train['HP'].unique()

array([30., 20., 10., 50.])

## Train

In [45]:
abod = ABOD()
abod.fit(train)

ABOD(contamination=0.1, method='fast', n_neighbors=5)

## Predict

In [18]:
test

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type
0,2.51,53.28,0.7,32.54,2662.0,69.58,3.48,0
1,2.66,55.24,0.7,34.45,2819.0,71.32,3.57,0
2,1.72,42.74,0.7,22.23,1819.0,60.21,3.01,0
3,2.20,49.15,0.7,28.50,2332.0,65.91,3.30,0
4,2.06,47.28,0.7,26.67,2182.0,64.24,3.21,0
...,...,...,...,...,...,...,...,...
7384,2.12,48.08,0.7,27.45,2246.0,64.96,3.25,7
7385,1.48,39.63,0.7,19.19,1570.0,57.44,2.87,7
7386,1.56,40.61,0.7,20.15,1649.0,58.32,2.92,7
7387,1.59,40.99,0.7,20.52,1679.0,58.66,2.93,7


In [46]:
num = 0

for col_1 in cols:
  for col_2 in cols:
    test['col_mul_'+str(num)] = test[col_1] * test[col_2]
    test['col_div_'+str(num)] = test[col_1] / test[col_2]
    num += 1

In [47]:
pred = abod.predict(test)

In [48]:
list(pred).count(0), list(pred).count(1)

(6728, 661)

## Submission

In [50]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI_SPARK/Data/answer_sample.csv')

In [52]:
submit['label'] = pred
submit.head()

,type,label
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [54]:
submit.to_csv('무지성.csv', index=False)